In [ ]:
import os, sys
import pandas as pd
import networkx as nx

In [ ]:
sys.path.insert(0, "./python")
import graph_preprocessing as gp

# Load parameters

In [ ]:
import sys
from datawand.parametrization import ParamHelper

In [ ]:
ph = ParamHelper('..', 'DataProcessing', sys.argv)

In [ ]:
experiment_id = "centrality"
snapshot_id = ph.get("snapshot_id")
amount_sat = ph.get("amount_sat")
day_interval = ph.get("day_interval")
drop_disabled = ph.get("drop_disabled")
drop_low_cap = ph.get("drop_low_cap")

In [ ]:
data_dir = ph.get("data_dir")
output_dir = "%s/simulations_%idays/%s/%s" % (data_dir, day_interval, snapshot_id, experiment_id)
print(output_dir)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2.) Load data

In [ ]:
snapshots = pd.read_csv("%s/directed_graphs/directed_temporal_multi_edges_%idays.csv" % (data_dir, day_interval))

In [ ]:
snapshots["snapshot_id"].min(), snapshots["snapshot_id"].max()

In [ ]:
edges = snapshots[snapshots["snapshot_id"]==snapshot_id]

In [ ]:
len(edges)

In [ ]:
filtered_edges = gp.prepare_edges_for_simulation(edges, amount_sat, drop_disabled, drop_low_cap)

In [ ]:
filtered_edges.head()

In [ ]:
G = nx.from_pandas_edgelist(filtered_edges, source="src", target="trg", edge_attr=["capacity", "total_fee"], create_using=nx.DiGraph())

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
centralities = {}

In [ ]:
%%time
centralities["degree"] = dict(G.degree())

In [ ]:
%%time
centralities["total_capacity"] = dict(G.degree(weight="capacity"))

In [ ]:
%%time
centralities["total_fee"] = dict(G.degree(weight="total_fee"))

In [ ]:
%%time
centralities["pagerank"] = dict(nx.pagerank(G))

In [ ]:
%%time
centralities["betweeness"] = dict(nx.betweenness_centrality(G))

In [ ]:
centrality_df = pd.DataFrame(centralities)

In [ ]:
centrality_df = centrality_df.reset_index().rename({"index":"node"}, axis=1)

In [ ]:
centrality_df.to_csv("%s/scores.csv" % output_dir, index=False)